In [85]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import matplotlib.pyplot as plt
import polars as pl
from datetime import datetime, time

from ts.supplier import TradeTick
from ts.supplier import TickSupplier

from ts.supplier import Bar
from ts.supplier import BarSupplier
from ts.supplier import BarAggregation
from ts.supplier import BarFeatureSupplier, BarFeatures
from ts.supplier import RollingFeaturesSupplier, SupplierType
from ts.supplier import Function, BaseSupplier
from ts.supplier import MultiplexSupplier

filepath = "/Users/jacobgora/projects/data/continuous_futures/CME-HO.parquet"
tick_supplier = TickSupplier(instrument="CME-HO")
tick_supplier.from_parquet(filepath)

In [87]:
supplier = MultiplexSupplier(suppliers=
    [
        BarFeatureSupplier(
            supplier=BarSupplier(
                supplier=tick_supplier,
                bar_aggregation=BarAggregation.TIME_MINUTES,
                size=60
            )
        ),
        BarFeatureSupplier(
            supplier=BarSupplier(
                supplier=tick_supplier,
                bar_aggregation=BarAggregation.TIME_MINUTES,
                size=1
            )
        )
    ]
)

In [88]:
rolling_feat_supplier = RollingFeaturesSupplier(
    functions=[Function.Z_SCORE],
    supplier=supplier,
    type_attributes=[
        BarFeatures.INTERNAL_BAR_STRENGTH,
        BarFeatures.OFI,
    ]
)

type_attr ='internal_bar_strength'
type_attr ='ofi'


In [89]:
print([e for e in rolling_feat_supplier.data.columns if "rolling_features" in e])

['rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-internal_bar_strength-z_score-10', 'rolling_features-bar_features-bar-CME-HO-time_minutes_agg-60-internal_bar_strength-z_score-10', 'rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ofi-z_score-10', 'rolling_features-bar_features-bar-CME-HO-time_minutes_agg-60-ofi-z_score-10']


In [106]:
barfeat_supplier = BarFeatureSupplier(
    supplier=BarSupplier(
        supplier=tick_supplier,
        bar_aggregation=BarAggregation.TIME_MINUTES,
        size=60
    )
)

/Users/jacobgora/.local/lib/python3.10/site-packages/numpy/ctypeslib.py:526: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return asarray(obj)


In [107]:
barfeat_supplier.bar_features

['bar_features-bar-CME-HO-time_minutes_agg-60-RETURN_TIMEDELTA',
 'bar_features-bar-CME-HO-time_minutes_agg-60-BID_SIZE_TIMEDELTA',
 'bar_features-bar-CME-HO-time_minutes_agg-60-ASK_SIZE_TIMEDELTA',
 'bar_features-bar-CME-HO-time_minutes_agg-60-RETURN_BID_SIZE',
 'bar_features-bar-CME-HO-time_minutes_agg-60-RETURN_ASK_SIZE',
 'bar_features-bar-CME-HO-time_minutes_agg-60-VOLUME',
 'bar_features-bar-CME-HO-time_minutes_agg-60-VOLUME_DELTA',
 'bar_features-bar-CME-HO-time_minutes_agg-60-CUMULATIVE_VOLUME_DELTA',
 'bar_features-bar-CME-HO-time_minutes_agg-60-INTERNAL_BAR_STRENGTH',
 'bar_features-bar-CME-HO-time_minutes_agg-60-OFI',
 'bar_features-bar-CME-HO-time_minutes_agg-60-OFI_NORMALIZED',
 'bar_features-bar-CME-HO-time_minutes_agg-60-OPEN_HIGH',
 'bar_features-bar-CME-HO-time_minutes_agg-60-OPEN_LOW',
 'bar_features-bar-CME-HO-time_minutes_agg-60-OPEN_CLOSE',
 'bar_features-bar-CME-HO-time_minutes_agg-60-SIGNAL',
 'bar_features-bar-CME-HO-time_minutes_agg-60-POS_REALIZED_VARIANCE',
 